In [1]:
# General imports
import numpy as np
import torch
import matplotlib.pyplot as plt
# DeepMoD stuff
from deepymod import DeepMoD
from deepymod.model.func_approx import NN
from deepymod.model.library import Library1D
from deepymod.model.constraint import LeastSquares
from deepymod.model.sparse_estimators import Threshold, PDEFIND
from deepymod.training import train
from deepymod.training.sparsity_scheduler import TrainTestPeriodic
from deepymod.data import Dataset
from deepymod.data.burgers import BurgersDelta

#if torch.cuda.is_available():
#    device = 'cuda'
#else:
device = 'cpu'

# Settings for reproducibility
np.random.seed(44)
torch.manual_seed(0)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False

# Making dataset

noise = 0.01
A = 1
v = 0.25

In [2]:
runs = 1
dataset = Dataset(BurgersDelta, A=A, v=v)

x = np.linspace(-4.1, 4.1, 120) 
t = np.linspace(0.1, 1.1, 100) 
t_grid, x_grid = np.meshgrid(t, x, indexing='ij')
X, y = dataset.create_dataset(x_grid.reshape(-1, 1), t_grid.reshape(-1, 1), n_samples=0, noise=noise, random=False, normalize=False)
X.shape

torch.Size([12000, 2])

In [3]:
xt = X.reshape(x_grid.shape[0],x_grid.shape[1],2)
yt = y.reshape(x_grid.shape[0],x_grid.shape[1])

In [4]:
for n_x in np.array([18]):
    number_of_samples = n_x
    ytrain = np.empty([yt.shape[0],number_of_samples])
    xtrain = np.empty([xt.shape[0],number_of_samples,xt.shape[2]])
    for i in np.arange(yt.shape[0]):
        idx = np.random.permutation(yt.shape[1])
        ytrain[i,:] = yt[i,idx][:number_of_samples]
        xtrain[i,:,0] = xt[i,idx,0][:number_of_samples]
        xtrain[i,:,1] = xt[i,idx,1][:number_of_samples]
    xtrain_b = np.transpose(xtrain,axes=(1,0,2))
    ytrain_b = ytrain.T
    X = torch.tensor(xtrain_b.reshape(-1,2), dtype=torch.float32, requires_grad=True)
    y = torch.tensor(ytrain_b.reshape(-1,1), dtype=torch.float32)
    print(y.shape)
    X, y = X.to(device), y.to(device)
    network = NN(2, [30, 30, 30, 30], 1)
    library = Library1D(poly_order=2, diff_order=3) # Library function
    estimator = Threshold(0.2) # Sparse estimator 
    constraint = LeastSquares() # How to constrain
    model = DeepMoD(network, library, estimator, constraint).to(device) # Putting it all in the model

    sparsity_scheduler = TrainTestPeriodic(periodicity=50, patience=500, delta=1e-7) # in terms of write iterations
    optimizer = torch.optim.Adam(model.parameters(), betas=(0.99, 0.99), amsgrad=True, lr=1e-3) # Defining optimizer

    train(model, X, y, optimizer, sparsity_scheduler, log_dir='runs/random_grid'+str(n_x), split=0.8, write_iterations=25, max_iterations=100000, delta=1e-4, patience=500) 
    print(model.constraint_coeffs(sparse=True, scaled=True))

torch.Size([1800, 1])
 44625  MSE: 8.06e-06  Reg: 6.19e-06  L1: 1.53e+00 Algorithm converged. Writing model to disk.
[tensor([[ 0.0000],
        [ 0.0000],
        [ 0.9093],
        [ 0.0000],
        [ 0.0000],
        [-0.6174],
        [ 0.0000],
        [ 0.0000],
        [ 0.0000],
        [ 0.0000],
        [ 0.0000],
        [ 0.0000]], grad_fn=<MulBackward0>)]


In [ ]:
for n_x in np.array([4,6,8,10,12,14,16]):
    for run in np.arange(runs):
        
        number_of_samples = n_x * 100
        idx = np.random.permutation(y.shape[0])
        X = torch.tensor(X[idx, :][:number_of_samples], dtype=torch.float32, requires_grad=True)
        y = torch.tensor(y[idx, :][:number_of_samples], dtype=torch.float32)
        X, y = X.to(device), y.to(device)
        network = NN(2, [30, 30, 30, 30], 1)
        library = Library1D(poly_order=2, diff_order=3) # Library function
        estimator = Threshold(0.2) # Sparse estimator 
        constraint = LeastSquares() # How to constrain
        model = DeepMoD(network, library, estimator, constraint).to(device) # Putting it all in the model

        sparsity_scheduler = TrainTestPeriodic(periodicity=50, patience=500, delta=1e-7) # in terms of write iterations
        optimizer = torch.optim.Adam(model.parameters(), betas=(0.99, 0.99), amsgrad=True, lr=1e-3) # Defining optimizer

        train(model, X, y, optimizer, sparsity_scheduler, log_dir='runs/rand'+str(n_x), split=0.8, write_iterations=25, max_iterations=100000, delta=1e-4, patience=500) 
        print(model.constraint_coeffs(sparse=True, scaled=True))

 28500  MSE: 8.57e-06  Reg: 4.27e-06  L1: 1.41e+00 Algorithm converged. Writing model to disk.
[tensor([[ 0.0000],
        [ 0.0000],
        [ 0.6382],
        [ 0.0000],
        [ 0.0000],
        [-0.7670],
        [ 0.0000],
        [ 0.0000],
        [ 0.0000],
        [ 0.0000],
        [ 0.0000],
        [ 0.0000]], device='cuda:0', grad_fn=<MulBackward0>)]
 99975  MSE: 6.56e-06  Reg: 9.61e-07  L1: 3.85e+00 Algorithm converged. Writing model to disk.
[tensor([[ 7.9732e-04],
        [-3.5464e-02],
        [ 6.7906e-01],
        [ 6.0199e-02],
        [ 2.7272e-01],
        [-8.8315e-01],
        [-3.8879e-01],
        [-2.4706e-01],
        [-4.5990e-01],
        [ 2.9216e-01],
        [ 2.1149e-01],
        [ 3.2241e-01]], device='cuda:0', grad_fn=<MulBackward0>)]
 36300  MSE: 1.05e-05  Reg: 4.87e-06  L1: 1.46e+00 